# 一键生成评价指标

In [51]:
import os
import pandas as pd
import numpy as np
import json

date = "0501"


gt_path = "/Users/wenchienyueh/Desktop/53_patients_real.csv"
folder_path = "/Users/wenchienyueh/Desktop/ex_tool_2_result"
res_folder_list = os.listdir(folder_path)
# res_folder_list = ['ex_clinical_8', 'ex_clinical_2', 'ex_clinical_7', 'ex_tool_2', 'ex_base_4', 'ex_base_3', 'ex_base_1', 'ex_one_2', 'ex_tool_1', 'ex_one_5', 'ex_clinical_4', 'ex_base_5', 'ex_clinical_3', 'ex_tool_4', 'ex_clinical_5', 'ex_one_4', 'ex_clinical_6', 'ex_clinical_1', 'ex_tool_3', 'ex_base_6', 'ex_base_2', 'ex_one_3']

print(len(res_folder_list))
print(res_folder_list)

with open("./train_list.txt", "r") as f:
    train_list = f.readlines()
train_all_list = [x.strip() for x in train_list]

with open("./test_list.txt", "r") as f:
    test_list = f.readlines()
test_all_list = [x.strip() for x in test_list]
print(len(train_all_list))
print(len(test_all_list))

4
['.DS_Store', 'time_consumption.json', 'abnormal_list.txt', 'axSpA_res.csv']
359
186


In [54]:
def cal_acc(res_df, gt_df, patient_id_list=None):
    cal_res_dict = {"tp_list": [], "fp_list": [], "tn_list": [], "fn_list": [], "p_unsure": [], "n_unsure": [], "tp": 0, "fp": 0, "tn": 0, "fn": 0, "p_unsure_num": 0, "n_unsure_num": 0}
    
    if patient_id_list is None:
        for i in range(len(res_df)):
            patient_id = res_df.loc[i, "patient_id"]
            pred_i = res_df.loc[i, "pred"]
            gt_i = gt_df[gt_df["patient_id"] == patient_id]["diagnosis_result"].values[0]
            if pred_i == -1:
                if gt_i == 1:
                    cal_res_dict["p_unsure_num"] += 1
                    cal_res_dict["p_unsure"].append(patient_id)
                else:
                    cal_res_dict["n_unsure_num"] += 1
                    cal_res_dict["n_unsure"].append(patient_id)
            elif pred_i == 0:
                if gt_i == 0:
                    cal_res_dict["tn"] += 1
                    cal_res_dict["tn_list"].append(patient_id)
                elif gt_i == 1:
                    cal_res_dict["fn"] += 1
                    cal_res_dict["fn_list"].append(patient_id)
            elif pred_i == 1:
                if gt_i == 0:
                    cal_res_dict["fp"] += 1
                    cal_res_dict["fp_list"].append(patient_id)
                elif gt_i == 1:
                    cal_res_dict["tp"] += 1
                    cal_res_dict["tp_list"].append(patient_id)

    else:
      for patient_id in patient_id_list:
    # 预测值处理
        hit = res_df[res_df["patient_id"] == patient_id]
        if hit.empty:
            print(f"❌ [PRED] patient_id {patient_id} 不在 res_df 中，预测设为 -1")
            pred_i = -1
        else:
            pred_i = hit["pred"].values[0]

        # ground truth 处理
        gt_row = gt_df[gt_df["patient_id"] == patient_id]
        if gt_row.empty:
            print(f"⚠️ [GT] patient_id {patient_id} 不在 gt_df 中，跳过")
            continue
        gt_i = gt_row["diagnosis_result"].values[0]

        # 分类统计逻辑
        if pred_i == -1:
            if gt_i == 1:
                cal_res_dict["p_unsure_num"] += 1
                cal_res_dict["p_unsure"].append(patient_id)
            else:
                cal_res_dict["n_unsure_num"] += 1
                cal_res_dict["n_unsure"].append(patient_id)
        elif pred_i == 0:
            if gt_i == 0:
                cal_res_dict["tn"] += 1
                cal_res_dict["tn_list"].append(patient_id)
            elif gt_i == 1:
                cal_res_dict["fn"] += 1
                cal_res_dict["fn_list"].append(patient_id)
        elif pred_i == 1:
            if gt_i == 0:
                cal_res_dict["fp"] += 1
                cal_res_dict["fp_list"].append(patient_id)
            elif gt_i == 1:
                cal_res_dict["tp"] += 1
                cal_res_dict["tp_list"].append(patient_id)
    return cal_res_dict
res_train_dict = cal_acc(res_df, gt_df, train_all_list)
print("✅ res_train_dict:", res_train_dict)

❌ [PRED] patient_id A027939 不在 res_df 中，预测设为 -1
⚠️ [GT] patient_id A027939 不在 gt_df 中，跳过
❌ [PRED] patient_id A0034905 不在 res_df 中，预测设为 -1
⚠️ [GT] patient_id A0034905 不在 gt_df 中，跳过
❌ [PRED] patient_id F417375 不在 res_df 中，预测设为 -1
⚠️ [GT] patient_id F417375 不在 gt_df 中，跳过
❌ [PRED] patient_id K0587923 不在 res_df 中，预测设为 -1
⚠️ [GT] patient_id K0587923 不在 gt_df 中，跳过
❌ [PRED] patient_id Y4044092 不在 res_df 中，预测设为 -1
⚠️ [GT] patient_id Y4044092 不在 gt_df 中，跳过
❌ [PRED] patient_id A0055452 不在 res_df 中，预测设为 -1
⚠️ [GT] patient_id A0055452 不在 gt_df 中，跳过
❌ [PRED] patient_id Y4490364 不在 res_df 中，预测设为 -1
⚠️ [GT] patient_id Y4490364 不在 gt_df 中，跳过
❌ [PRED] patient_id Y1503382 不在 res_df 中，预测设为 -1
⚠️ [GT] patient_id Y1503382 不在 gt_df 中，跳过
❌ [PRED] patient_id Y2605879 不在 res_df 中，预测设为 -1
⚠️ [GT] patient_id Y2605879 不在 gt_df 中，跳过
❌ [PRED] patient_id A0048819 不在 res_df 中，预测设为 -1
⚠️ [GT] patient_id A0048819 不在 gt_df 中，跳过
❌ [PRED] patient_id Y2707920 不在 res_df 中，预测设为 -1
⚠️ [GT] patient_id Y2707920 不在 gt_df 中，跳过
❌ [

In [48]:
metric_df = pd.DataFrame(columns=["folder_name", "split", "tp", "fp", "tn", "fn", "p_unsure_num", "n_unsure_num", "mean_time"])
res_folder_list = [""] 
print("res_folder_list:", res_folder_list)
print("Total folders found:", len(res_folder_list))

for folder in res_folder_list:

    res_folder_path = os.path.join(folder_path, folder)
    res_path = os.path.join(res_folder_path, "axSpA_res.csv")
    time_path = os.path.join(res_folder_path, "time_consumption.json")
    abnormal_list_path = os.path.join(res_folder_path, "abnormal_list.txt")
    print("="*40)
    print(f"[{folder}] res_path: {res_path}")
    res_df = pd.read_csv(res_path)
    print(f"[{folder}] res_df shape: {res_df.shape}")
    print(f"[{folder}] res_df columns: {res_df.columns.tolist()}")
    print("="*40)
    with open(abnormal_list_path, "r") as f:
        abnormal_list = f.read().split("\n")
        
    time_consumption_dict = json.load(open(time_path, "r"))
    res_df = pd.read_csv(res_path)
    gt_df = pd.read_csv(gt_path)
    res_df["pred"] = 0
    for i in range(len(res_df)):
        res = str(res_df.loc[i, "res"])  # 👈 强制转换为字符串
        if "未诊断为axSpA" in res:
            res_df.loc[i, "pred"] = 0
        elif "可以诊断为axSpA" in res:
            res_df.loc[i, "pred"] = 1
        else:
            res_df.loc[i, "pred"] = -1
    missing_abnormal_ids = []
    for abnormal_id in abnormal_list:
        if abnormal_id == "":
            continue
        gt_row = gt_df[gt_df["patient_id"] == abnormal_id]
        if gt_row.empty:
            missing_abnormal_ids.append(abnormal_id)
            continue
        gt_pred = gt_row["diagnosis_result"].values[0]
    print("以下 abnormal_list 中的 ID 没出现在 ground truth 中：")
    print(missing_abnormal_ids)
    print(folder, len(res_df), len(gt_df))
    res_train_dict = cal_acc(res_df, gt_df, train_all_list)
    res_test_dict = cal_acc(res_df, gt_df, test_all_list)

    res_dict = cal_acc(res_df, gt_df)


    mean_time = np.mean(list(time_consumption_dict.values()))

    metric_df.loc[len(metric_df)] = [folder, "all", res_dict["tp"], res_dict["fp"], res_dict["tn"], res_dict["fn"], res_dict["p_unsure_num"], res_dict["n_unsure_num"], mean_time]

    metric_df.loc[len(metric_df)] = [folder, "train", res_train_dict["tp"], res_train_dict["fp"], res_train_dict["tn"], res_train_dict["fn"], res_train_dict["p_unsure_num"], res_train_dict["n_unsure_num"], mean_time]

    metric_df.loc[len(metric_df)] = [folder, "test", res_test_dict["tp"], res_test_dict["fp"], res_test_dict["tn"], res_test_dict["fn"], res_test_dict["p_unsure_num"], res_test_dict["n_unsure_num"], mean_time]

metric_df["unsure_num"] = metric_df["p_unsure_num"] + metric_df["n_unsure_num"]

metric_df["acc"] = (metric_df["tp"] + metric_df["tn"]) / (metric_df["tp"] + metric_df["tn"] + metric_df["fp"] + metric_df["fn"])
metric_df["sensitivity"] = metric_df["tp"] / (metric_df["tp"] + metric_df["fn"])
metric_df["specificity"] = metric_df["tn"] / (metric_df["tn"] + metric_df["fp"])
metric_df["precision"] = metric_df["tp"] / (metric_df["tp"] + metric_df["fp"])
metric_df["recall"] = metric_df["tp"] / (metric_df["tp"] + metric_df["fn"])
metric_df["f1"] = 2 * metric_df["precision"] * metric_df["recall"] / (metric_df["precision"] + metric_df["recall"])
metric_df["b_acc"] = (metric_df["sensitivity"] + metric_df["specificity"]) / 2

metric_df["un_sure_rate"] = (metric_df["p_unsure_num"] + metric_df["n_unsure_num"]) / (metric_df["tp"] + metric_df["tn"] + metric_df["fp"] + metric_df["fn"] + metric_df["p_unsure_num"] + metric_df["n_unsure_num"])


res_folder_list: ['']
Total folders found: 1
[] res_path: /Users/wenchienyueh/Desktop/ex_tool_2_result/axSpA_res.csv
[] res_df shape: (53, 3)
[] res_df columns: ['patient_id', 'res', 'diagnosis_result']
以下 abnormal_list 中的 ID 没出现在 ground truth 中：
[]
 53 53


IndexError: index 0 is out of bounds for axis 0 with size 0

In [19]:
metric_df.to_csv("./res_process/metric_"+date+".csv", index=False)

In [20]:
metric_df

,folder_name,split,tp,fp,tn,fn,p_unsure_num,n_unsure_num,mean_time,unsure_num,acc,sensitivity,specificity,precision,recall,f1,b_acc,un_sure_rate


# 生成各自的文件

In [5]:
import os
import pandas as pd
import numpy as np
import json

date = "0501"


gt_path = "/data1/lzf/301agent/data/agent_sheet_all_0415.csv"
folder_path = "./results"
res_folder_list = os.listdir(folder_path)
# res_folder_list = ['ex_base_1', 'ex_base_2', 'ex_base_3', 'ex_base_4', 'ex_base_5', 'ex_base_6', 'ex_clinical_1', 'ex_clinical_2', 'ex_clinical_3', 'ex_clinical_4', 'ex_clinical_5', 'ex_clinical_7', 'ex_one_2', 'ex_one_3', 'ex_tool_1', 'ex_tool_2', 'ex_tool_3', 'ex_tool_4']
# res_folder_list = ['ex_base_1', 'ex_base_2', 'ex_base_3', 'ex_base_4', 'ex_base_5', 'ex_base_6', 'ex_clinical_1', 'ex_clinical_2', 'ex_clinical_3', 'ex_clinical_4', 'ex_one_2', 'ex_one_3', 'ex_tool_1', 'ex_tool_2', 'ex_tool_3', 'ex_tool_4']
print(len(res_folder_list))
print(res_folder_list)

with open("./train_list.txt", "r") as f:
    train_list = f.readlines()
train_all_list = [x.strip() for x in train_list]

with open("./test_list.txt", "r") as f:
    test_list = f.readlines()
test_all_list = [x.strip() for x in test_list]
print(len(train_all_list))
print(len(test_all_list))


29
['ex_clinical_8', 'ex_clinical_2', 'ex_all_1', 'ex_clinical_7', 'ex_tool_2', 'ex_all_2', 'ex_base_4', 'ex_base_3', 'ex_base_1', 'ex_one_2', 'ex_tool_1', 'ex_one_5', 'ex_all_4', 'ex_all_3', 'ex_clinical_4', 'ex_base_9', 'ex_base_5', 'ex_one_6', 'ex_clinical_3', 'ex_tool_4', 'ex_clinical_5', 'ex_one_4', 'ex_clinical_6', 'ex_clinical_1', 'ex_tool_3', 'ex_base_6', 'ex_base_2', 'ex_one_3', 'ex_memory_1']
359
186


In [6]:
sorted(res_folder_list)

['ex_all_1',
 'ex_all_2',
 'ex_all_3',
 'ex_all_4',
 'ex_base_1',
 'ex_base_2',
 'ex_base_3',
 'ex_base_4',
 'ex_base_5',
 'ex_base_6',
 'ex_base_9',
 'ex_clinical_1',
 'ex_clinical_2',
 'ex_clinical_3',
 'ex_clinical_4',
 'ex_clinical_5',
 'ex_clinical_6',
 'ex_clinical_7',
 'ex_clinical_8',
 'ex_memory_1',
 'ex_one_2',
 'ex_one_3',
 'ex_one_4',
 'ex_one_5',
 'ex_one_6',
 'ex_tool_1',
 'ex_tool_2',
 'ex_tool_3',
 'ex_tool_4']

In [46]:
for folder in res_folder_list:

    res_folder_path = os.path.join(folder_path, folder)
    res_path = os.path.join(res_folder_path, "axSpA_res.csv")
    time_path = os.path.join(res_folder_path, "time_consumption.json")
    abnormal_list_path = os.path.join(res_folder_path, "abnormal_list.txt")
    print("="*40)
    print(f"[{folder}] res_path: {res_path}")
    res_df = pd.read_csv(res_path)
    print(f"[{folder}] res_df shape: {res_df.shape}")
    print(f"[{folder}] res_df columns: {res_df.columns.tolist()}")
    print("="*40)
    with open(abnormal_list_path, "r") as f:
        abnormal_list = f.read().split("\n")
        
    time_consumption_dict = json.load(open(time_path, "r"))
    res_df = pd.read_csv(res_path)
    gt_df = pd.read_csv(gt_path)
    res_df["pred"] = 0
    for i in range(len(res_df)):
        res = str(res_df.loc[i, "res"])  #  强制转换为字符串
        if "未诊断为axSpA" in res:
            res_df.loc[i, "pred"] = 0
        elif "可以诊断为axSpA" in res:
            res_df.loc[i, "pred"] = 1
        else:
            res_df.loc[i, "pred"] = -1
    missing_abnormal_ids = []

    for abnormal_id in abnormal_list:
        if abnormal_id == "":
            continue
        gt_row = gt_df[gt_df["patient_id"] == abnormal_id]
        if gt_row.empty:
            missing_abnormal_ids.append(abnormal_id)
            continue
        gt_pred = gt_row["diagnosis_result"].values[0]
        ...

    print("以下 abnormal_list 中的 ID 没出现在 ground truth 中：")
    print(missing_abnormal_ids)

    res_df["label"] = np.nan
    missing_label_ids = []
    for i in range(len(res_df)):
        pid = res_df.loc[i, "patient_id"]
        gt_row = gt_df[gt_df["patient_id"] == pid]
        if gt_row.empty:
            missing_label_ids.append(pid)
            continue
        gt_i = gt_row["diagnosis_result"].values[0]
        res_df.loc[i, "label"] = gt_i
    if missing_label_ids:
        print("⚠️ 以下 patient_id 未能在 gt_df 中找到 diagnosis_result：")
        print(missing_label_ids)    
    res_df.to_csv("./res_process/csv_res/" + folder + ".csv", index=False)

[] res_path: /Users/wenchienyueh/Desktop/ex_tool_2_result/axSpA_res.csv
[] res_df shape: (53, 3)
[] res_df columns: ['patient_id', 'res', 'diagnosis_result']
以下 abnormal_list 中的 ID 没出现在 ground truth 中：
['K0204047', 'Y3625818']
⚠️ 以下 patient_id 未能在 gt_df 中找到 diagnosis_result：
['Y9561794', 'K0623670', 'Y6237327', 'K1057828', 'Y9746979', 'Y4906457', 'K1012366', '594569', 'V0157072', 'Y9634338', 'V0231491', 'K0850349', 'Y7834604', 'K0478607', 'Y9713134', 'Y3696865', 'Y9006659', 'V0309587', 'K0479027', 'V0051545', 'Y9050905', 'K0676707', 'F428384', 'Y8874894', 'K0826068', 'Y4829325', 'K1066505', 'V0333013', 'Y5357602', 'Y9369974', 'A0013372', 'V0042172', 'K1069142', 'Y9639235', 'K0668936', 'K0455050', 'K0871598', 'V0087990', 'K0818982', 'Y5690202', 'Y1468182', 'Y9693644', 'K0880621', 'K0912680', 'Y9567751', 'Y9803283', 'Y9784084', 'A250663', 'K1041027', 'F459639', 'Y9951596', 'V0142489', 'Y9862738']
